Useful resources:

https://jazz-soft.net/demo/GeneralMidi.html
https://midi.org/about-midi-part-3midi-messages


https://audiodev.blog/midi-note-chart/midi-note-chart.jpg

In [1]:
from mido import Message, MidiFile, MidiTrack
import random
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(Message('program_change', program=0, time=0))

# "W, W, H, W, W, W, H" for major scales
# "W, H, W, W, H, W, W" for minor scales


allowed_notes = [60,62,64,65,67,69,71, 72] # C Major

allowed_notes_f_minor = [65, 67, 68, 70, 72, 73, 75, 77]

# 50 is fast but still somewhat recognizable as music

# 10 seems bugged when playing in VLC with Arachno soundfont
time_distance = 50


for i in range(500):
    random_note = random.choice(allowed_notes_f_minor)
    track.append(Message('note_on', note=random_note, velocity=64, time=0))
    track.append(Message('note_off', note=random_note, velocity=127, time=time_distance))

    #Random instrument change
    #track.append(Message('program_change', program=random.randint(0,128), time=0))


mid.save('new_song.mid')

In [2]:
# Get the MIDI notes (piano keys) corresponding to the major / minor scale based on the key (provided as an int as in https://audiodev.blog/midi-note-chart/midi-note-chart.jpg)


def get_major_scale_from_note(key : int, octaves : int) -> list[int]:
    notes = [key]
    current_base = key
    for _ in range(octaves):
        notes.extend([current_base+2,current_base+4,current_base+5,current_base+7,current_base+9,current_base+11, current_base+12])
        current_base+=12
    return notes

print(get_major_scale_from_note(60,1))

def get_minor_scale_from_note(key : int, octaves : int) -> list[int]:
    notes = [key]
    current_base = key
    for _ in range(octaves):
        notes.extend([current_base+2,current_base+3,current_base+5,current_base+7,current_base+8,current_base+10,current_base+12])
        current_base+=12
    return notes

print(get_minor_scale_from_note(65,1))

[60, 62, 64, 65, 67, 69, 71, 72]
[65, 67, 68, 70, 72, 73, 75, 77]


In [10]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(Message('program_change', program=0, time=0))
extended_scale = get_minor_scale_from_note(65,3)
for i in range(500):
    random_note = random.choice(extended_scale)
    track.append(Message('note_on', note=random_note, velocity=64, time=0))
    track.append(Message('note_off', note=random_note, velocity=127, time=time_distance))

    #Random instrument change
    #track.append(Message('program_change', program=random.randint(0,128), time=0))


mid.save('new_song_extended_range.mid')

### The above functions gets 8 notes when asked for 1 octave, for more you get 15 etc (not even numbers)

8 notes - 3 bits
16 notes - 4 bits





In [11]:
def get_major_scale_16_notes(base_key : int) -> list[int]:
    notes = [base_key]
    current_base = base_key
    for _ in range(2):
        notes.extend([current_base+2,current_base+4,current_base+5,current_base+7,current_base+9,current_base+11, current_base+12])
        current_base+=12
    notes.append(current_base+2)
    return notes

print(get_major_scale_16_notes(60))

# "W, W, H, W, W, W, H" for major scales
# "W, H, W, W, H, W, W" for minor scales

def get_minor_scale_16_notes(base_key : int) -> list[int]:
    notes = [base_key]
    current_base = base_key
    for _ in range(2):
        notes.extend([current_base+2,current_base+3,current_base+5,current_base+7,current_base+8,current_base+10,current_base+12])
        current_base+=12
    notes.append(current_base+2)
    return notes

print(get_minor_scale_16_notes(65))

[60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79, 81, 83, 84, 86]
[65, 67, 68, 70, 72, 73, 75, 77, 79, 80, 82, 84, 85, 87, 89, 91]


# 16 notes - 4 bits of data
2 notes is 8 bits, so 1 ASCII character

In [12]:
ord("a")

97

In [13]:
chr(97)

'a'

In [73]:
"Is this ascii?".isascii()

True

In [74]:
"is Źdźbło ascii?".isascii()

False

### Velocity can be used to add more bits of info

Identical timings (time between note on / off)
Note - 4 bits

16 velocities to choose from - 4 bits
(or 4 *note on* velocities and 4 *note off* velocities?)

In [17]:
#velocities = [64,80,96,112] # this didn't sound good
velocities = [64,65,66,67]

def get_random_midi_song(scale : list[int], number_of_notes : int):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    for i in range(number_of_notes):
        random_note = random.choice(scale)
        track.append(Message('note_on', note=random_note, velocity=random.choice(velocities), time=0))
        track.append(Message('note_off', note=random_note, velocity=random.choice(velocities), time=time_distance))
        #Random instrument change
        #track.append(Message('program_change', program=random.randint(0,128), time=0))
    return mid



get_random_midi_song(get_minor_scale_from_note(65,1), 200).save('song_with_velocity_changes.mid')

In [62]:
def int_to_note_info(data : int) -> list[int]:
    """
    Converts number in the range 0-255 into three numbers : note value (4 bits), note_on_velocity (2 bits), note_off_velocity (2 bits)
    """
    if data>255:
        raise ValueError("The provided int data is higher than 255")
    bits = f'{data:08b}'
    note_on_velocity = int(bits[:2], 2)
    note_off_velocity = int(bits[2:4], 2)
    note_value = int(bits[4:], 2)
    return [note_value, note_on_velocity, note_off_velocity]


def note_info_to_int(note : int, note_on_velocity : int, note_off_velocity : int) -> int:
    """
    Converts data from int_to_note_info back to a single number in the range 0-255
    """
    data = f'{note_on_velocity:02b}{note_off_velocity:02b}{note:04b}'
    data = int(data, 2)
    return data




print(int_to_note_info(8))
print(note_info_to_int(8,0,0))

[8, 0, 0]
8


In [75]:
for x in range(256):
    note_info = int_to_note_info(x)
    original_number = note_info_to_int(*note_info)
    #print(f"{x} || {original_number}")
    assert x==original_number

In [67]:
def ascii_numbers_from_string(input_string : str) -> list[int]:
    if not input_string.isascii():
        raise ValueError("The provided string is not ASCII")
    return [ord(x) for x in input_string]

def encode_string_to_midi(data : str, musical_key: list[int], filename : str ="encoded.mid"):
    time_distance = 50
    data_numerical = ascii_numbers_from_string(data)
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    velocities = [64,65,66,67]
    for data_point in data_numerical:
        note_index, note_on_velocity_index, note_off_velocity_index = int_to_note_info(data_point)
        note_value = musical_key[note_index]
        track.append(Message('note_on', note=note_value, velocity=velocities[note_on_velocity_index], time=0))
        track.append(Message('note_off', note=note_value, velocity=velocities[note_off_velocity_index], time=time_distance))
    mid.save(filename)

In [68]:
def decode_string_from_midi(file_path : str, musical_key: list[int])-> str:
    velocities = [64,65,66,67]
    mid = MidiFile(file_path)
    data_numeric = []
    for msg in mid:
        if msg.type == 'note_on':
            note_value = msg.note
            note_on_velocity = msg.velocity
        if msg.type == 'note_off':
            note_off_velocity = msg.velocity
            data_numeric.append(note_info_to_int(musical_key.index(note_value), velocities.index(note_on_velocity), velocities.index(note_off_velocity)))
            note_value = None
            note_on_velocity = None
            note_off_velocity = None
    data_string = ''.join([chr(x) for x in data_numeric])
    return data_string

In [69]:
encode_string_to_midi("Test data after encoding", get_minor_scale_16_notes(65))
decode_string_from_midi('encoded.mid', get_minor_scale_16_notes(65))

'Test data after encoding'

In [72]:
bee_movie = """According to all known laws of aviation, there is no way a bee should be able to fly.
Its wings are too small to get its fat little body off the ground.
The bee, of course, flies anyway because bees don't care what humans think is impossible.
Yellow, black. Yellow, black. Yellow, black. Yellow, black.
Ooh, black and yellow!
Let's shake it up a little.
Barry! Breakfast is ready!
Coming!
Hang on a second.
Hello?
Barry?
Adam?
Can you believe this is happening?
I can't.
I'll pick you up.
Looking sharp.
Use the stairs, Your father paid good money for those.
Sorry. I'm excited.
Here's the graduate.
We're very proud of you, son.
A perfect report card, all B's.
Very proud.
Ma! I got a thing going here.
You got lint on your fuzz.
Ow! That's me!
Wave to us! We'll be in row 118,000.
Bye!
Barry, I told you, stop flying in the house!
Hey, Adam.
Hey, Barry.
Is that fuzz gel?
A little. Special day, graduation.
Never thought I'd make it.
Three days grade school, three days high school.
Those were awkward.
Three days college. I'm glad I took a day and hitchhiked around The Hive.
You did come back different.
Hi, Barry. Artie, growing a mustache? Looks good.
Hear about Frankie?
Yeah."""

encode_string_to_midi(bee_movie, get_minor_scale_16_notes(65), "bee movie.mid")
print(decode_string_from_midi('bee movie.mid', get_minor_scale_16_notes(65)))

According to all known laws of aviation, there is no way a bee should be able to fly.
Its wings are too small to get its fat little body off the ground.
The bee, of course, flies anyway because bees don't care what humans think is impossible.
Yellow, black. Yellow, black. Yellow, black. Yellow, black.
Ooh, black and yellow!
Let's shake it up a little.
Barry! Breakfast is ready!
Coming!
Hang on a second.
Hello?
Barry?
Adam?
Can you believe this is happening?
I can't.
I'll pick you up.
Looking sharp.
Use the stairs, Your father paid good money for those.
Sorry. I'm excited.
Here's the graduate.
We're very proud of you, son.
A perfect report card, all B's.
Very proud.
Ma! I got a thing going here.
You got lint on your fuzz.
Ow! That's me!
Wave to us! We'll be in row 118,000.
Bye!
Barry, I told you, stop flying in the house!
Hey, Adam.
Hey, Barry.
Is that fuzz gel?
A little. Special day, graduation.
Never thought I'd make it.
Three days grade school, three days high school.
Those were awkw